In [2]:
#need to read in a layer
#test a probe and write the results 

#multiple layers? --> last three of hte xl embeddings 

In [1]:
import pandas as pd
df = pd.read_csv('./data/animals_true_false.csv')
source_labels = df['label']

In [5]:
import os

# Specify the directory path
directory_path = "data/embeddings/llama3/basic"

# Get all files in the directory
all_files = os.listdir(directory_path)

for filename in all_files:
    # Check if the filename contains square brackets
    if "[" in filename or "]" in filename:
        # Construct the old and new file paths
        old_path = os.path.join(directory_path, filename)
        new_filename = filename.replace("[", "").replace("]", "")
        new_path = os.path.join(directory_path, new_filename)

        # Rename the file
        os.rename(old_path, new_path)
        print(f"Renamed {filename} to {new_filename}")

Renamed [285]:-1.npy to 285:-1.npy
Renamed [402]:-2.npy to 402:-2.npy
Renamed [295]:-1.npy to 295:-1.npy
Renamed [412]:-2.npy to 412:-2.npy
Renamed [257]:-2.npy to 257:-2.npy
Renamed [247]:-2.npy to 247:-2.npy
Renamed [924]:-3.npy to 924:-3.npy
Renamed [11]:-3.npy to 11:-3.npy
Renamed [131]:-2.npy to 131:-2.npy
Renamed [934]:-3.npy to 934:-3.npy
Renamed [121]:-2.npy to 121:-2.npy
Renamed [902]:-1.npy to 902:-1.npy
Renamed [37]:-1.npy to 37:-1.npy
Renamed [764]:-2.npy to 764:-2.npy
Renamed [790]:-3.npy to 790:-3.npy
Renamed [774]:-2.npy to 774:-2.npy
Renamed [780]:-3.npy to 780:-3.npy
Renamed [27]:-1.npy to 27:-1.npy
Renamed [912]:-1.npy to 912:-1.npy
Renamed [969]:-2.npy to 969:-2.npy
Renamed [677]:-1.npy to 677:-1.npy
Renamed [188]:-2.npy to 188:-2.npy
Renamed [811]:-2.npy to 811:-2.npy
Renamed [979]:-2.npy to 979:-2.npy
Renamed [801]:-2.npy to 801:-2.npy
Renamed [667]:-1.npy to 667:-1.npy
Renamed [198]:-2.npy to 198:-2.npy
Renamed [651]:-3.npy to 651:-3.npy
Renamed [729]:-3.npy to 72

In [2]:
import os
import numpy as np

# Set the folder path
folder_path = "data/embeddings/llama3"

merged_embeddings = []
basic_embeddings = []
llama_embeddings = []

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    
    # Check if the file is a numpy file
    if filename.endswith(".npy"):
        # Load the embedding data from the file
        embedding_data = np.load(file_path)
        
        # Check if the filename contains "merged"
        if "merged" in filename:
            merged_embeddings.append(embedding_data)
        else:
            basic_embeddings.append(embedding_data)

In [3]:
from itertools import combinations

def generate_combinations(x, y):
    """
    Generates all combinations of numbers from x to y.
    
    Args:
        x (int): The lower bound of the range (inclusive).
        y (int): The upper bound of the range (inclusive).
        
    Returns:
        list: A list of tuples, where each tuple represents a combination
        of numbers from x to y.
    """
    nums = list(range(x, y+1))  # Generate a list of numbers from x to y
    combinations_list = []
    
    # Generate combinations of different lengths
    for length in range(1, len(nums)+1):
        combs = combinations(nums, length)
        combinations_list.extend(combs)
    
    return combinations_list

In [29]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

def probe(embeddings, labels, layer_list, input_dim=1600):
  X = np.array(embeddings)  
  y = np.array(labels)

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

  # Define and compile model
  model = Sequential()
  model.add(Dense(64, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(16, activation='relu'))
  model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))  # Assuming binary classification
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  # Train the model
  model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

  # Evaluate the model
  test_loss, test_acc = model.evaluate(X_test, y_test)

  layer_str = '-'.join(map(str, layer_list))
  save_data(test_loss, test_acc, layer_str)

def save_data(test_loss, test_acc, layer):
  write_data = f'{layer}: {test_loss} {test_acc}'
  with open('./statistics/Layer_Data/basic_manual_testing.txt', 'a') as file:
    file.write(f'{write_data}\n')

In [19]:
indices = generate_combinations(0,48)

In [16]:
indices

[(47,), (48,), (47, 48)]

In [7]:
def parse_embeddings(embeddings_list, index_list):
    flat_list = []
    for index in index_list:
        flat_list.extend(embeddings_list[index])
    return flat_list  

In [25]:
indices = [(0,47)]

In [30]:
for i in range(10):
  for index_list in indices:
    #construct embeddings for this list 
    #create the label amount by multiplying 
    embeddings = parse_embeddings(basic_embeddings, index_list)
    labels = df['label'] * len(index_list)
    probe(embeddings=embedding_data, labels=labels, layer_list=index_list)

Epoch 1/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.1308 - loss: 15.2419 - val_accuracy: 0.0000e+00 - val_loss: -7.9447
Epoch 2/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0925 - loss: -0.4587 - val_accuracy: 0.0000e+00 - val_loss: -6.5257
Epoch 3/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0607 - loss: 47.1940 - val_accuracy: 0.0000e+00 - val_loss: -6.1647
Epoch 4/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1160 - loss: 6.6016 - val_accuracy: 0.0000e+00 - val_loss: -4.0008
Epoch 5/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0459 - loss: 10.5144 - val_accuracy: 0.0000e+00 - val_loss: -7.8474
Epoch 6/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0443 - loss: -4.6846 - val_accuracy: 0.0000e+00 - val_loss: -15.6850
Epoch 7/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0247 - loss: 24.0240 - val_accuracy: 0.0000e+00 - val_loss: -16.9968
Epoch 8/10
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.0312 - loss